In [ ]:
import pyspark
from pyspark.sql import SparkSession

# if you change congig do spark.stop (the last step) and then restart this

sc_conf = pyspark.SparkConf()
sc_conf.setAppName("spark-sample-app")
sc_conf.set('spark.executor.cores','1')
sc_conf.set("spark.executor.memory", '1g')
#sc_conf.set("spark.sql.files.maxPartitionBytes", '134217728 ')
#sc_conf.set("spark.sql.shuffle.partitions", '2')

spark = SparkSession.builder.config(conf=sc_conf).getOrCreate()

In [ ]:
def printRdd(df):
       print(df.rdd.toDebugString().decode("utf-8")) 

In [ ]:
df = spark.read.json("file:/bdpc/data/input/samples/lab05.json")

ids = df.select(df.data.id.alias("id"))
ids.show(5)

ids.explain()

printRdd(ids)

In [ ]:
from pyspark.sql.functions import count

groupingExpr = (ids.id).alias("group")
q = (ids
     .groupBy(groupingExpr)
     .agg(count(ids.id).alias("count")))

q.explain()

printRdd(q)

q.write.mode("overwrite").json("/bdpc/data/lab05/json-id/")

In [ ]:
from pyspark.sql.functions import count

groupingExpr = (ids.id).alias("group")
q = (ids
     .repartition(groupingExpr)
     .groupBy(groupingExpr)
     .agg(count(ids.id).alias("count")))

q.explain()

printRdd(q)

q.write.mode("overwrite").json("/bdpc/data/lab05/json-id/")

# hadoop fs -ls /bdpc/data/lab05/json-id/

In [ ]:
from pyspark.sql.functions import count

groupingExpr = (ids.id).alias("group")
q = (ids
     .repartition(4, groupingExpr)
     .groupBy(groupingExpr)
     .agg(count(ids.id).alias("count")))

q.explain()

printRdd(q)

q.write.mode("overwrite").json("/bdpc/data/lab05/json-id/")

# hadoop fs -ls /bdpc/data/lab05/json-id/

In [ ]:
spark.stop()